In [15]:
import pandas as pd
import numpy as np
from pathlib import Path

In [16]:
financial_features_df = pd.read_csv(Path('../Data/financial_features.csv'), infer_datetime_format=True)
news_features_df = pd.read_csv(Path('../Data/news_features.csv'), infer_datetime_format=True)
target_df = pd.read_csv(Path('../Data/target.csv'), infer_datetime_format=True)

In [17]:
model_data_df = target_df.merge(news_features_df, on=['date', 'symbol'], how='left').merge(financial_features_df, on=['date', 'symbol'], how='left')
model_data_df = model_data_df.set_index(['date', 'symbol'])
model_data_df

close  compound_b  neg_b  neu_b  pos_b  compound_t  \
date       symbol                                                          
2020-05-18 AZN    -0.043409         NaN    NaN    NaN    NaN         NaN   
           JNJ    -0.050989         NaN    NaN    NaN    NaN         NaN   
           MRNA    6.896897         NaN    NaN    NaN    NaN         NaN   
           PFE     0.217015         NaN    NaN    NaN    NaN         NaN   
2020-05-19 AZN     0.008815         NaN    NaN    NaN    NaN         NaN   
...                     ...         ...    ...    ...    ...         ...   
2021-09-16 PFE    -0.303450         NaN    NaN    NaN    NaN         NaN   
2021-09-17 AZN    -0.710924         NaN    NaN    NaN    NaN         NaN   
           JNJ    -0.168846         NaN    NaN    NaN    NaN         NaN   
           MRNA   -0.909216         NaN    NaN    NaN    NaN         NaN   
           PFE    -0.524806         NaN    NaN    NaN    NaN         NaN   

                   neg_t  neu_t  pos_t  lagged_returns  5_day_returns  \
date       symbol                                                       
2020-05-18 AZN       NaN    NaN    NaN             NaN            NaN   
           JNJ       NaN    NaN    NaN             NaN            NaN   
           MRNA      NaN    NaN    NaN             NaN            NaN   
           PFE       NaN    NaN    NaN             NaN            NaN   
2020-05-19 AZN       NaN    NaN    NaN             NaN            NaN   
...                  ...    ...    ...             ...            ...   
2021-09-16 PFE       NaN    NaN    NaN        0.419327      -0.673629   
2021-09-17 AZN       NaN    NaN    NaN       -0.333846      -0.368960   
           JNJ       NaN    NaN    NaN       -0.753556      -0.381085   
           MRNA      NaN    NaN    NaN       -0.195201      -0.803230   
           PFE       NaN    NaN    NaN       -0.957801      -0.721896   

                   volume_change  100_day_ma  50_day_ema   z_score  \
date       symbol                                                    
2020-05-18 AZN               NaN         NaN         NaN       NaN   
           JNJ               NaN         NaN         NaN       NaN   
           MRNA              NaN         NaN         NaN       NaN   
           PFE               NaN         NaN         NaN       NaN   
2020-05-19 AZN               NaN         NaN         NaN       NaN   
...                          ...         ...         ...       ...   
2021-09-16 PFE         -0.471603   -0.392229   -0.513749  0.241581   
2021-09-17 AZN          0.368177   -0.307709   -0.737815 -0.383384   
           JNJ          1.786586    1.276303   -0.743223 -0.672622   
           MRNA         2.709334    1.189807    0.898473  0.841928   
           PFE          1.994574    0.953955   -0.616018  0.130724   

                   percentile_transformation  stochastic_k  stochastic_d  
date       symbol                                                         
2020-05-18 AZN                           NaN           NaN           NaN  
           JNJ                           NaN           NaN           NaN  
           MRNA                          NaN           NaN           NaN  
           PFE                           NaN           NaN           NaN  
2020-05-19 AZN                           NaN           NaN           NaN  
...                                      ...           ...           ...  
2021-09-16 PFE                      1.621139     -0.688645      0.490815  
2021-09-17 AZN                     -1.262149     -0.635936     -0.026989  
           JNJ                      0.740614     -0.112492     -1.416905  
           MRNA                     1.741995      1.239606      0.486527  
           PFE                      1.690200     -0.709394      0.414015  

[1352 rows x 18 columns]

In [18]:
model_data_df['compound'] = model_data_df['compound_b'] + model_data_df['compound_t']
model_data_df['neg'] = model_data_df['neg_b'] + model_data_df['neg_t']
model_data_df['neu'] = model_data_df['neu_b'] + model_data_df['neu_t']
model_data_df['pos'] = model_data_df['pos_b'] + model_data_df['pos_t']
model_data_df = model_data_df.reset_index()
model_data_df = model_data_df.drop(columns=['compound_b', 'compound_t', 'neg_b', 'neg_t', 'neu_b', 'neu_t', 'pos_b', 'pos_t'])

In [19]:
model_data_df = model_data_df.set_index(['date', 'symbol']).dropna()
X_f = model_data_df[['lagged_returns', '5_day_returns', 'volume_change', '100_day_ma', '50_day_ema', 'z_score', 'percentile_transformation', 'stochastic_k', 'stochastic_d']]
X_n = model_data_df[['compound', 'pos', 'neu', 'neg']]
y = model_data_df['close']

In [20]:
X_f

lagged_returns  5_day_returns  volume_change  100_day_ma  \
date       symbol                                                             
2020-10-06 AZN           0.272739      -0.553073      -0.785842   -0.579927   
           MRNA         -0.570753      -0.087483       0.015920   -0.863756   
           PFE          -0.547654      -0.192224       0.280476   -0.253446   
2020-10-07 AZN          -0.660355      -0.521417      -0.218159   -0.613925   
           JNJ          -0.213417      -0.287633      -0.029664   -0.251519   
...                           ...            ...            ...         ...   
2021-09-14 JNJ          -0.092924      -0.812107      -0.383203   -0.260698   
           MRNA         -0.309774      -0.474759      -0.237337   -0.067622   
           PFE           0.591506      -0.812059      -0.644279   -0.306512   
2021-09-15 MRNA          0.042338       0.196058      -0.326323   -0.223411   
           PFE           0.474485      -0.723635       0.109899   -0.125491   

                   50_day_ema   z_score  percentile_transformation  \
date       symbol                                                    
2020-10-06 AZN      -0.658956 -1.173585                  -0.916845   
           MRNA     -0.088272 -0.352326                   0.498901   
           PFE      -0.570574 -0.757660                   0.084536   
2020-10-07 AZN      -0.677117 -1.246575                  -0.890178   
           JNJ      -0.440344 -0.481961                   1.639772   
...                       ...       ...                        ...   
2021-09-14 JNJ      -0.775098 -0.637909                  -1.641984   
           MRNA      1.064046  0.909406                   1.690200   
           PFE      -0.472043  0.305015                  -0.312563   
2021-09-15 MRNA      1.143372  0.929343                   1.741995   
           PFE      -0.460126  0.304982                   1.552078   

                   stochastic_k  stochastic_d  
date       symbol                              
2020-10-06 AZN        -0.508184     -1.468156  
           MRNA       -0.175037      0.660828  
           PFE        -0.835190      0.338031  
2020-10-07 AZN        -0.510836     -1.499814  
           JNJ         1.385632      0.040768  
...                         ...           ...  
2021-09-14 JNJ        -0.114878     -1.424301  
           MRNA        1.226539      0.469501  
           PFE        -0.690705      0.417210  
2021-09-15 MRNA        1.262540      0.511256  
           PFE        -0.688471      0.458220  

[808 rows x 9 columns]

In [21]:
from sklearn.base import clone

def model_generator(X, y, algo):
    dates = X

    models = pd.Series(index=X.index)

    features = X
    target = y

    for date in dates:
        X_train = features.loc()[None: date]
        y_train = target.loc()[None: date]

        X_train.fillna(0)
        y_train.fillna(0)

        model = clone(algo)
        model.fit(X_train, y_train)
        models.loc[date] = model

    begin_dates = models.index
    end_dates = models.index[1:].append(pd.to_datetime(['2021-10-08']))

    predictions = pd.Series(index=features.index)
    for i, model in enumerate(models):
        X = features.xs(slice(begin_dates[i], end_dates[i]), level='date', drop_level=False)
        prediction = pd.Series(model.predict(X), index=X.index)
        predictions.loc[X.index] = prediction

    return models, predictions

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

lr_models, lr_predictions = model_generator(X_f, y, algo=LinearRegression())
rf_models, rf_predictions = model_generator(X_n, y, algo=RandomForestClassifier())    

C:\Users\ayres\miniconda3\envs\machineLearning\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


ValueError: Found array with 0 sample(s) (shape=(0, 9)) while a minimum of 1 is required.

In [ ]:
from sklearn.linear_model import LassoCV

def ensemble_data(X, y):
    X_with_y = X.join(y).replace({np.inf:None, -np.inf:None}).dropna()
    X = X_with_y.iloc[:,:-1]
    y = X_with_y.iloc[:,-1]
    return X, y

X_ensemble, y_ensemble = ensemble_data(X=pd.concat([lr_predictions.rename('linear'), rf_predictions.rename('forest')], axis=1,), y=y)

ensemble_models, ensemble_predictions = model_generator(X_ensemble, y_ensemble, LassoCV())

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error

def calc_scorecard(y_pred,y_true):
    
    def make_df(y_pred,y_true):
        y_pred.name = 'y_pred'
        y_true.name = 'y_true'

        df = pd.concat([y_pred,y_true],axis=1).dropna()

        df['sign_pred'] = df.y_pred.apply(np.sign)
        df['sign_true'] = df.y_true.apply(np.sign)
        df['is_correct'] = 0
        df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
        df['is_incorrect'] = 0
        df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
        df['is_predicted'] = df.is_correct + df.is_incorrect
        df['result'] = df.sign_pred * df.y_true 
        return df
    
    df = make_df(y_pred,y_true)
    
    scorecard = pd.Series()
    scorecard.loc['RSQ'] = r2_score(df.y_true,df.y_pred)
    scorecard.loc['MAE'] = mean_absolute_error(df.y_true,df.y_pred)
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    scorecard.loc['edge_to_noise'] = scorecard.loc['edge'] / scorecard.loc['noise']
    scorecard.loc['edge_to_mae'] = scorecard.loc['edge'] / scorecard.loc['MAE']
    return scorecard  

In [24]:
pd.DataFrame([model.coef_ for model in ensemble_models], columns=X_ensemble.columns,index=ensemble_models.index).plot()

NameError: name 'ensemble_models' is not defined

In [ ]:
score_ens = calc_scorecard(y_pred=ensemble_predictions,y_true=y_ensemble).rename('LassoEnsemble')
score_linear = calc_scorecard(y_pred=lr_predictions,y_true=y_ensemble).rename('LinearRegression')
score_tree = calc_scorecard(y_pred=rf_predictions,y_true=y_ensemble).rename('RandomForest')


scores = pd.concat([score_linear,score_tree,score_ens],axis=1)
scores.loc['edge_to_noise'].plot.bar(color='blue',legend=True)
scores.loc['edge'].plot(color='red',legend=True)
scores.loc['noise'].plot(color='green',legend=True)

plt.show()
print(scores)